In [0]:
# %% [markdown]
# This will be simple kernel showcasing some analysis of the Titanic competetion dataset. We will first begin by importing the data and some packages. I will also be using a multiplotter function that will make comparison graphs easier to view.

# %% [code] {"_execution_state":"idle"}
## Importing packages

# This R environment comes with all of CRAN and many other helpful packages preinstalled.
# You can see which packages are installed by checking out the kaggle/rstats docker image: 
# https://github.com/kaggle/docker-rstats

library(tidyverse) # metapackage with lots of helpful functions
library(data.table) # for reading the data using fread
library(randomForest) # for modelling
library(mice) # for filling in missing information
library(ggthemes) # for some plot themes
library(Rmisc) # for multiplot function
library(scales) # for graphs
## Running code

# In a notebook, you can run a single code cell by clicking in the cell and then hitting 
# the blue arrow to the left, or by clicking in the cell and pressing Shift+Enter. In a script, 
# you can run code by highlighting the code you want to run and then clicking the blue arrow
# at the bottom of this window.

## Reading in files

# You can access files from datasets you've added to this kernel in the "../input/" directory.
# You can see the files added to this kernel by running the code below. 

list.files(path = "../input")
train <- fread('../input/titanic/train.csv',stringsAsFactors = F)
test <- fread('../input/titanic/test.csv', stringsAsFactors = F)
## Saving data

# If you save any files or images, these will be put in the "output" directory. You 
# can see the output directory by committing and running your kernel (using the 
# Commit & Run button) and then checking out the compiled version of your kernel.

# %% [markdown]
# First, lets take a look at our data. There are 891 rows for the training dataset, and 418 rows for the test dataset. There is a missing column for the test dataset, which contains whether or not the passenger Survived. So, simply add a new column (test$Survived) with all zeros and then combine the two sets. This will make it easier to mutate or change the way certain variables appear, and so lead to better predictions. It's also convienient to remember which part of the dataset is 'Train' and which is 'Test' and, while it could be done using the rows to identify them (1:891 is train and the rest is test), my preference is to use another column. But that doesn't really matter. Once that is done, the data is combined into a new dataframe named all. 

# %% [code]
dim(train)
dim(test)
head(test)
head(train)


test$Survived <- 0
train$Set <- 'Train'
test$Set <- 'Test'
dim(train)
dim(test)

all<- rbind(train,test)
dim(all)

# %% [markdown]
# **First Look At DATA - Completeness**
# 
# Lets begin by checking the data for completeness. There are 264 rows with incomplete data. The most common missing value is Age. Age will be dealt with later using the mice library. 
# 
# 

# %% [code]
all[!complete.cases(all),]
table(all$Cabin =='')

# %% [markdown]
# One of the caveats of using complete.cases is that if a string is '', it is counted as complete. Let's use a table to see where the cabin value is blank. That is quite a bit of empty strings.
# First, add a deck column so there is no need to worry about room number. Now, using this new column, lets try to approximate what deck passengers are in using the average fare for that deck.

# %% [code]


all$Deck <-""
all <-all %>% mutate (Deck = str_sub(Cabin,1,1))

table(all$Deck)


all %>% filter(Deck !='') %>%
  group_by(Deck) %>%
  dplyr::summarise(medianPrice = median(Fare,na.rm =T), meanPrice = mean(Fare,na.rm = T), minPrice = min(Fare,na.rm = T), maxPrice = max(Fare,na.rm =T))



# %% [markdown]
# Looking at the table above, its hard to find meaningful information due to the fact that there is quite a bit of overlap using median, mean, min, and max.

# %% [markdown]
# Before doing anything, lets see how Deck plays into whether or not a Passenger Survived. 
# 

# %% [code]
p1 <-all %>% group_by(Deck) %>% 
  ggplot(aes(x=Deck, fill=as.factor(Survived))) +
  geom_bar() + theme_few()

p2 <- all %>% filter(Deck !='') %>% group_by(Deck) %>% 
  ggplot(aes(x=Deck, fill=as.factor(Survived))) +
  geom_bar() + theme_few() + labs(fill = 'Survival Condition')

Rmisc::multiplot(p1,p2)


# %% [markdown]
# Looking at the graph, the blank deck dominates the scale, so lets try again after filtering it out. It does seem like you had a better chance of survival if you were some decks as opposed to others. Another thing to consider is how correlated Pclass could be with Deck. Higher socioeconomic class Passengers could be in better cabins.
# 

# %% [code]
table(all$Deck,all$Pclass)

# %% [markdown]
# Looking at the table, for now lets just leave the Deck field as is with the Unknowns being U. Because there is so much missing data, filling it could lead to some bias.

# %% [code]
all$Deck[all$Deck ==''] <- 'U'
table(all$Deck)

# %% [markdown]
# Going back to the table looking at the summary for each Deck, there seem to be cases where the Fare is 0 which is probably not actually the case. There is also 1 passenger with Fare = NA. These passengers will be treated the same when calculating their Fare. Lets take a look at what values influence Fare. The values that intuitively influence Fare are Pclass and Sex. Age also does but with so many NA values, its best to not include it in our calculations. Another possible value would be the group size. If there are a lot family members travelling together, they could get a discount. This could be explored in the future but for now, the focus will be on Pclass, Sex and Embarked. 
# 

# %% [code]
all[all$Fare==0,]
table(is.na(all$Fare))

all %>% group_by(Pclass) %>%
 dplyr::summarise(meanP = mean(Fare,na.rm= T), medianP = median(Fare,na.rm= T),)


all %>% group_by(Sex) %>%
 dplyr::summarise(meanP = mean(Fare,na.rm= T), medianP = median(Fare,na.rm= T),)


all %>% group_by(Embarked) %>% filter(Embarked != '') %>%
  ggplot(aes(x=Embarked, y=Fare)) + geom_boxplot() + scale_y_continuous(labels=dollar_format())




# %% [markdown]
# Looking at the boxplot, there does seem to be some correlation between Embarked and Fare, with the the higher Fares being paid by passengers departing from C. Lets create a comparison table.

# %% [code]
all %>% group_by(Pclass,Sex,Embarked) %>%
 dplyr::summarise(meanP = mean(Fare,na.rm= T), medianP = median(Fare,na.rm= T),)

# %% [markdown]
# Looking at the table above, there is a strong case that the female passengers who has a blank embarked field, are from port C or from port S. While this wasn't something that was being looked for, its one less thing to think about. 

# %% [code]
all[all$Embarked=='',]




# %% [markdown]
# There are two passengers who share the same Pclass, Fare, and Sex. The median values for C and S are similar so lets test again using only passengers who survived.

# %% [code]
all %>% filter(Survived ==1, Sex =='female', Pclass==1) %>%group_by(Pclass,Sex,Embarked) %>%
 dplyr::summarise(meanP = mean(Fare,na.rm= T), medianP = median(Fare,na.rm= T),)

# %% [markdown]
# A case could be made for the Passengers being from either C or S but since the median of S is closer to the Fare of the two passengers, I will assign them to port S.

# %% [code]
all[all$Embarked=='',]$Embarked <- 'S'


# %% [markdown]
# Back to Fare, There will be some hardcoding here because I'm not sure on how to go about automating this process. All the Passengers seem to be from port S and are male, so lets use the median Fare for Male passengers from Port S adjusting for each Pclass and fill the 0 values using that.

# %% [code]
all[all$Fare==0,]
all %>% filter(Embarked=='S',Sex=='male') %>%
    group_by(Pclass) %>%
    dplyr::summarise(meanFare = mean(Fare,na.rm =T), medianFare = median(Fare,na.rm = T))
all[all$Fare==0 &all$Pclass==1,]$Fare <- 35.50
all[all$Fare==0 &all$Pclass==2,]$Fare <- 13.00
all[all$Fare==0 &all$Pclass==3,]$Fare <- 8.05

all[all$Fare==0,]

# %% [markdown]
# Now there are no longer any zeroes for the Fare value but there is still that 1 NA value. Lets take a glance at that row. It seems that the passenger is a male from Port S in Pclass 3. Fortunately, the median for a passenger with those conditions was already calculated.

# %% [code]
all[is.na(Fare),]
all[is.na(Fare),]$Fare <- 8.05

# %% [markdown]
# **Second Look at Data - Mutating, Etc**
# 
# At this point, other than Age, all NA values should have been dealt with. Cabin will remain as is, so the blanks there don't really matter. Lets start cleaning up our data to make it easier to build the model. The Sex column will be converted to 1s and 0s where 0s are males. The Deck column will also be converted to numbers where A=1, B=2,...,T=8, and U=0.

# %% [code]
all <- all %>% mutate(Sex = case_when(Sex =='male'~0,Sex=='female'~1),
               Deck = case_when(Deck=='A'~1, Deck=='B'~2,
                                Deck=='C'~3, Deck=='D'~4,
                                Deck=='E'~5, Deck=='F'~6,
                                Deck=='G'~7, Deck=='T'~8,
                                Deck=='U'~0,))
str(all)


head(all,10)

# %% [markdown]
# Taking a look at the Name column, all the Passengers have a title that precedes their Name. There is also the Surname portion which could help when grouping passengers into Families. Lets do some string mutation using regex to isolate what we want.

# %% [code]
all$title <-  gsub('(.*, )|(\\..*)', '', all$Name)

head(all,10)

table(all$title)

# %% [markdown]
# Looking at the table above, there are a lot of titles worn by less than 10 people. The titles will just be grouped into a title called 'Other'.

# %% [code]
otherTitles <- c('Capt','Col','Dona','Don','Dr','Jonkheer','Lady','Major','Mlle','Mme','Ms','Rev','Sir','the Countess')


all$title[all$title %in% otherTitles] <- 'Other'
table(all$title)

# %% [markdown]
# Following that, lets see whether or not family size has an impact on Survival.
# First, create a new column whose value is equal to the sum of SibSp and Parch +1. This will state how many family members are there for a passenger. Then, the families will be classified
# into groups based on their size. A famiily size of 1 is an individual, size >1 and <= 4 will be medium, and anything greater than 4 will be large.

# %% [code]
all$Fsize <- all$SibSp + all$Parch +1
all <- all %>%
  mutate(FGroup = case_when(Fsize ==1~'Single',Fsize >1 & Fsize <=4~'Med',
                            Fsize >4~'Large'))

table(all$FGroup)

# %% [markdown]
# Lastly, the Age column. There are 263 missing values in this column and it could be major factor in survival. This column will be filled in using the 'mice' package. 'mice' stands for Multivariate Inputation using Chained Equations and can be used to fill in missing values. First lets make sure how many missing values we have in the dateset.

# %% [code]
md.pattern(all)

# %% [markdown]
# The above graph tells us that there 1046 rows where there are 0 missing fields, and 263 rows where there is a missing value (Age). Before mice is applied to the data, lets factorize some variables that should be factors. These include Survived, Deck, Fsize, FGroup, Title, Sex, Pclass ,and Embarked. Then, set a random seed to make sure the data is replicable.
# 

# %% [code]
set.seed(1234)

all <-all %>% dplyr::mutate(Survived = as.factor(Survived),
                       Deck = as.factor(Deck),
                       Fsize = as.factor(Fsize),
                       FGroup = as.factor(FGroup),
                       title = as.factor(title),
                       Sex = as.factor(Sex),
                       Pclass = as.factor(Pclass),
                       Embarked = as.factor(Embarked)) %>%
    dplyr::rename(Title = title)


ignoredCols <- c('Name','Cabin','Ticket','FGroup','Survived')
ind <- mice(all[,!names(all) %in% ignoredCols], method = 'rf')

miceComplete <- complete(ind)

# %% [markdown]
# * So now that the mice imputations are done, lets just do a quick comparison of the ages generated by mice and the ones that were in our data. This will be by just looking at the density of the data.

# %% [code]
p1 <-all %>% filter(!is.na(Age)) %>%
ggplot(aes(x=Age)) +
geom_density(col='blue')
p2 <- miceComplete %>%
ggplot(aes(x=Age)) +
geom_density(col='green')
Rmisc::multiplot(p1,p2)

# %% [markdown]
# It looks fine so its time to replace the Age field with the values genereated by mice. A quick check to make sure that our data is complete

# %% [code]
all$Age <- miceComplete$Age

md.pattern(all)

# %% [markdown]
# There are still some more columns that could be improved upon. Now that age is complete, it can be grouped into a factor column using ranges.
# 

# %% [code]

all <- all %>% mutate(
  AgeGroup = case_when(Age <13~'Child',Age>=13 & Age <20~'Teen',
                       Age >=20 & Age <30~'Young Adult',
                       Age >= 30 & Age <55~'Adult',Age >=55~'Senior')
)
table(all$AgeGroup)
all$AgeGroup <- as.factor(all$AgeGroup)


# %% [markdown]
# **Time to look at Some Graphs!**
# 
# Now that the data has been modified to fit some requirements, lets take a look at how Survived is influenced by other variables.
# First, Age group.

# %% [code]

all %>%
  ggplot(aes(AgeGroup,fill=Survived))+
  geom_bar() +facet_grid(.~Sex,
                         labeller = as_labeller(c(`0` = "Male",  `1` = 'Female')) ) +
theme_minimal()

all %>%
  group_by(AgeGroup,Sex) %>%
  dplyr::summarise(Survivors = sum(as.numeric(as.character(Survived))), count =n(),
                   percentage = Survivors/count)

# %% [markdown]
# Across the board, if you were female, your survival odds went up by a lot. But lets try age group without separating the sexes, just to see what was the safest and least safe group to be in.

# %% [code]
all %>%
  ggplot(aes(AgeGroup,fill=Survived))+ geom_bar() +
  scale_fill_brewer(palette = 2) +  
theme_minimal()

all %>%
  group_by(AgeGroup) %>%
  dplyr::summarise(Survivors = sum(as.numeric(as.character(Survived))), count =n(),
                   percentage = Survivors/count)


# %% [markdown]
# Looking at the graph and the numbers, the safest group to be in was the child group. Oddly enough, it wasn't that much safer to be a Young Adult, compared to the least safe Senior group.
# Looking at the data with genders separate, the safest person on the ship seemed to be a Teenaged Girl. Conversely, if you were a Teenaged Male, the odds were not in your favor.
# 
# Following the age groups, lets move on to Pclass.

# %% [code]
all %>% group_by(Pclass) %>%
ggplot(aes(x=Pclass, fill=Survived)) + geom_bar()+
scale_fill_brewer(palette =6) +
facet_grid(.~Sex,labeller = as_labeller(c(`0` = "Male",  `1` = 'Female')) ) +
theme_minimal()

all %>%
  group_by(Pclass,Sex) %>%
  dplyr::summarise(Survivors = sum(as.numeric(as.character(Survived))), count =n(),
                   percentage = Survivors/count)

# %% [markdown]
# Looking at the split, the most deaths are from male passengers in Pclass 3. The least deaths are from the female passengers in Pclass in 1. 

# %% [code]
all %>%
  ggplot(aes(Pclass,fill=Survived))+ geom_bar() +
  scale_fill_brewer(palette = 6) +  
theme_minimal()

all %>%
  group_by(Pclass) %>%
  dplyr::summarise(Survivors = sum(as.numeric(as.character(Survived))), count =n(),
                   percentage = Survivors/count)

# %% [markdown]
# As a whole, the safest place to be was in P class 1. 

# %% [code]
all %>%
  ggplot(aes(Sex,fill=Survived))+ geom_bar() +
  scale_fill_brewer(palette = 7) +  
theme_minimal()


all %>%
  group_by(Sex) %>%
  dplyr::summarise(Survivors = sum(as.numeric(as.character(Survived))), count =n(),
                   percentage = Survivors/count)

# %% [markdown]
# Based on all the graphs seen, the statement 'Women and children first' was very true for the Titanic disaster. 50% of women passengers on the ship survived.
# 
# Lets just create a few more graphs using the Fsize, FGroup, Title, and Embarked
# 
# 
# Fsize

# %% [code]
all %>%
  ggplot(aes(Fsize,fill=Survived))+ geom_bar() +
  scale_fill_brewer(palette = 'Blues') +  
theme_few()



# %% [markdown]
# FGroup
# 
# 

# %% [code]
all %>%
  ggplot(aes(FGroup,fill=Survived))+ geom_bar() +
  scale_fill_brewer(palette = 'Greens') +  
theme_few()

all %>%
  group_by(FGroup) %>%
  dplyr::summarise(Survivors = sum(as.numeric(as.character(Survived))), count =n(),
                   percentage = Survivors/count)

# %% [markdown]
# Title

# %% [code]
all %>%
  ggplot(aes(Title,fill=Survived))+ geom_bar() +
  scale_fill_brewer(palette = 'PuBu') +  
theme_few()

all %>%
  group_by(Title) %>%
  dplyr::summarise(Survivors = sum(as.numeric(as.character(Survived))), count =n(),
                   percentage = Survivors/count)

# %% [markdown]
# Embarked

# %% [raw]
# Embarked

# %% [code]
all %>%
  ggplot(aes(Embarked,fill=Survived))+ geom_bar() +
  scale_fill_brewer(palette = 'Purples') +  
theme_few()

all %>%
  group_by(Embarked) %>%
  dplyr::summarise(Survivors = sum(as.numeric(as.character(Survived))), count =n(),
                   percentage = Survivors/count)

# %% [markdown]
# **Time to build our prediction model.**
# 
# We will be using randomForests for now. Maybe later, we can explore different models and their predictiveness.
# 
# 

# %% [code]
train <- all[all$Set=='Train',]
test <- all[all$Set=='Test',]

set.seed(111)
rfModel <- randomForest(Survived~Deck+Pclass+Fare+
                        Fsize+FGroup+Embarked+
                          Title+AgeGroup+Sex+ Age,
                        data= train)
rfModel


# %% [markdown]
# With our Model built, we can use it on our test set and then submit our answer.

# %% [code]
prediction <- predict(rfModel, test)
submission <- data.frame(PassengerID = test$PassengerId, Survived = prediction)